When running the LTOP workflow there is a circumstance where ties can occur in the paramater selection process. In these instances, version 0.1.0 is just selecting the first item in the df. However, for reproducability and to improve the param set pick, a more codified score/rank approach is required. This notebook is for working out the logic of that process before it is integrated into the existing workflow. This was developed in the ltop_py env on Islay. 

In [ ]:
import pandas as pd 
import os
import sys
import matplotlib.pyplot as plt 
import seaborn as sns 
import numpy as np 
%matplotlib inline 

In [ ]:
#read in the intermediate param selection outputs - this path can be changed to a local version
fn = r'/vol/v1/proj/LTOP_FTV_Py/param_selection_testing_outputs/intermediate_testing_output.csv'
df = pd.read_csv(fn)
df.columns

In [ ]:
####set some user args#### 
#select a cluster id, not sure if this is what we want to do or if we want to iterate? 
select_cluster = 37
clust_df = df.loc[df.cluster_id == select_cluster]
clust_df.columns
#select the param you want to plot 


In [19]:
def get_max_mean(df1,col_name): 
    '''
    Calculate the mean of the combined rankVscore and rankAICcscore, considering the weighting factors, 
    for each possible value for a given param. Then take the max mean value. 
    '''
    #this assumes you've already subset by cluster_id as is the case in the param selection code 
    #get the mean by possible param values 
    df1 = pd.DataFrame(df1.groupby([col_name])['combined'].mean()).reset_index()
    # df1 = df.sort_values('combined')
    # #get the max mean value for the given param 
    # df2 = df1.loc[df1['combined'] == df1['combined'].max()]
    return df1

In [35]:
#do the subsetting 
#first get the max mean value for each param - this will be a one line dataframe for the cluster id in question
#do recoveryThreshold first
#TODO do we want to calculate these means on the whole cluster_id? or just on the tied ones? 
rec_select = get_max_mean(clust_df,'recoveryThreshold')
#next do spikeThreshold
spike_select = get_max_mean(clust_df,'spikeThreshold')
#then maxSegments
max_select = get_max_mean(clust_df,'maxSegments')
#then pvalThreshold 
pval_select = get_max_mean(clust_df,'pvalThreshold')

clust_df['rec_rank'] = clust_df['recoveryThreshold'].map(dict(zip(rec_select.recoveryThreshold,rec_select.combined)))
clust_df['spike_rank'] = clust_df['spikeThreshold'].map(dict(zip(spike_select.spikeThreshold,spike_select.combined)))
clust_df['max_rank'] = clust_df['maxSegments'].map(dict(zip(max_select.maxSegments,max_select.combined)))
clust_df['pval_rank'] = clust_df['pvalThreshold'].map(dict(zip(pval_select.pvalThreshold,pval_select.combined)))

#then do the sequential subsetting, starting with the full dataframe - in the actual code this is a subset of the df for the cluster
# #TODO the clust_df here is not actually correct, that will be the ties not the full df for the cluster_id
clust_df.rec_rank.max()
df1 = clust_df.loc[clust_df.rec_rank == clust_df.rec_rank.max()]
df2 = df1.loc[df1.spike_rank == df1.spike_rank.max()]
df3 = df2.loc[df2.max_rank == df2.max_rank.max()]
df4 = df3.loc[df3.pval_rank == df3.pval_rank.max()]

df4.shape
# df2 = df1.loc[df1.spikeThreshold == spike_select.spikeThreshold.iloc[0]]
# df3 = df2.loc[df2.maxSegments == max_select.maxSegments.iloc[0]]
# df4 = df3.loc[df3.pvalThreshold == pval_select.pvalThreshold.iloc[0]]

#this will be the output, which ideally has only one row left? 



/home/ben.roberts/.conda/envs/ltop_py/lib/python3.7/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]
/home/ben.roberts/.conda/envs/ltop_py/lib/python3.7/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/home/ben.roberts/.conda/envs/ltop_py/lib/python3.7/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_inde

(5, 67)